<a href="https://colab.research.google.com/github/TeDand/university-chatbot/blob/main/emdeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("tweet_eval", "irony")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.0 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2862 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/irony/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import pandas as pd

df_train = pd.DataFrame(dataset['train'])
df_val = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
del df_train, df_val, df_test
df.head()

,text,label
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online @user #s...,1
3,@user @user So is he banded from wearing the c...,1
4,Just found out there are Etch A Sketch apps. ...,1


In [3]:
!pip install transformers
import pandas as pd
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, random_split
from transformers import BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import sys
import numpy as np
import time
import datetime

def tokenize_and_format(sentences):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sentence in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 64,           # Pad & truncate all sentences.
                          padding = 'max_length',
                          truncation = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )

      # Add the encoded sentence to the list.
      input_ids.append(encoded_dict['input_ids'])

      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])
  return input_ids, attention_masks

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import torch

# I already read my data in question 1
# df = pd.read_csv('my_data.csv')
# df = pd.read_csv('tweets.csv')

df = df.sample(frac=1).reset_index(drop=True)

texts = df.text.values
labels = df.label.values

### tokenize_and_format() is a helper function provided in helpers.py ###
input_ids, attention_masks = tokenize_and_format(texts)

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

Original:  What a golden morning. 😒 #not
Token IDs: tensor([ 101, 2054, 1037, 3585, 2851, 1012,  100, 1001, 2025,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])


In [11]:
query_sentence = 'I love ice-cream!'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

encoded_dict = tokenizer.encode_plus(
                    query_sentence,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 64,           # Pad & truncate all sentences.
                    padding = 'max_length',
                    truncation = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
              )

query_input_id = encoded_dict['input_ids']
query_attention_mask = encoded_dict['attention_mask']
print(query_input_id)

tensor([[ 101, 1045, 2293, 3256, 1011, 6949,  999,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])


In [19]:
from sklearn.neighbors import NearestNeighbors

k = 5  # Number of nearest neighbors to retrieve
knn_model = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model.fit(input_ids)

# Find the k nearest neighbors
distances, indices = knn_model.kneighbors(query_input_id)

# Retrieve the nearest sentences
nearest_sentences = [texts[idx] for idx in indices[0]]

print("Query Sentence:", query_sentence)
print("Nearest Sentences:", nearest_sentences)

Query Sentence: I love ice-cream!
Nearest Sentences: ['@user thanks for noticing me', '@user skiing in Regents Park much #not', 'I have such a loving family', 'I love when this happens', '@user Probably a Nigerian.']


Other possible similarity functions:

- Euclidean Distance:
  
  Instead of using cosine similarity, you can calculate the Euclidean distance between sentence embeddings using the euclidean_distances function from scikit-learn. Note that Euclidean distance is a dissimilarity metric, so you would need to invert the values to interpret them as similarities.

- Manhattan Distance:
  
  Manhattan distance (also known as L1 distance or city block distance) can be calculated using the manhattan_distances function from scikit-learn. Similar to Euclidean distance, you would need to invert the values to interpret them as similarities.

- Jaccard Similarity:

  If you want to focus on the overlapping tokens or words between sentences, you can use Jaccard similarity. First, convert sentences into sets of tokens and then compute the Jaccard similarity using the formula: intersection / union.

- Cosine Similarity with TF-IDF:

  Another option is to use cosine similarity with TF-IDF (Term Frequency-Inverse Document Frequency) vectors. You can create TF-IDF vectors for each sentence using the TfidfVectorizer from scikit-learn and then calculate cosine similarity using cosine_similarity from scikit-learn.

In [ ]:
# import nltk
# nltk.download('punkt')  # Download the necessary tokenizer data

# documents = web_scrape()  
# input_ids = []
# attention_masks = []

# for document in documents:
#   sentences = nltk.sent_tokenize(document)
#   new_input_ids, new_attention_masks = tokenize_and_format(sentences)

#   input_ids = input_ids + new_input_ids
#   attention_masks = attention_masks + new_attention_masks